In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Users/hoon/Library/Fonts/NanumSquareRegular.ttf').get_name()
plt.rc("font", family=font_name)
import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import time
import re

---
## (4). 전처리

- 머신러닝 데이터에서 필수적으로 해야할 전처리
    * 1. 결측치 무조건 확인 후 제거
    * 2. 데이터는 무조건 숫자로
    * 3. Feature Sclaing

#### 문자열 해결을 위한 데이터 인코딩
* 레이블 인코딩 : 문자를 숫자로
* 원-핫 인코딩 : 새로운 피처를 추가해 고유값에 해당하는 칼럼에만 1을 표시하고 나머지는 0으로 표시(사전에 미리 숫자로 변환되어있어야) // 입력값은 2차원 [[]]

In [ ]:
#### 1. 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

items = ["TV", "냉장고", "전자레인지", "컴퓨터", "선풍기", "선풍기", "믹서", "믹서"]
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items) # encoder.fit은 리스트 내 데이터값들의 숫자를 매칭해주는 것. encoder.transform은 던진 데이터 값을 매칭해 숫자로 반환해주는 것 

print("인코딩 변환값:", labels)
print("클래스 분류:", encoder.classes_) #여기서 encoder는 item 리스트가 fit되어있는 값.
print("디코딩 원본값:", encoder.inverse_transform([0, 1, 4, 5, 3, 3, 2, 2]))

#### 0,1,2,3,4 로 인코딩하면 발생하는 문제 : 순서나 크기로 인식해 분류 아닌 다른 의미로 분석할 수 있음


인코딩 변환값: [0 1 4 5 3 3 2 2]
클래스 분류: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']
디코딩 원본값: ['TV' '냉장고' '전자레인지' '컴퓨터' '선풍기' '선풍기' '믹서' '믹서']


In [ ]:
#### 2. 원핫 인코딩 : 레이블 인코딩으로 문자 -> 숫자로 바꾼 후 사용 가능
# 더미변수를 머신러닝 방식에 맞게 바꾸는 것. 와이드형으로 바꾼다고 생각하면 된다. 카테고리에 맞춰 칼럼이 늘어남
# 0001, 0010, 0100 등으로 바꿔 순서나 크기로 인식하지 않고 분류!하게끔 만드는 인코딩 방법.
# 머신러닝에선 단골 방식. 머신러닝 성능에 엄청난 도움이 된다.

# 새로운 피처를 추가해 고유값에 해당하는 칼럼에만 1을 표시하고 나머지는 0으로 표시
# 입력값은 2차원 [[]]

from sklearn.preprocessing import OneHotEncoder

items = ["TV", "냉장고", "전자레인지", "컴퓨터", "선풍기", "선풍기", "믹서", "믹서"]
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

#2차원 배열로 변환
labels = labels.reshape(-1, 1)
labels

encoder_one = OneHotEncoder()
encoder_one.fit(labels)
ot = encoder_one.transform(labels)

print("원핫인코딩데이터:")
print(ot.toarray())
print("크기:", ot.shape)

원핫인코딩데이터:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
크기: (8, 6)


In [ ]:
# get_dummies()
df = pd.DataFrame({'item' : ["TV", "냉장고", "전자레인지", "컴퓨터", "선풍기", "선풍기", "믹서", "믹서"]})
pd.get_dummies(df)

# 간편하다. 난 이거!
# df에 더미변수될 칼럼만 던져주면 된다.

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


#### (2). Feature Sclaing

+ 표준화, 이상치 맞춰주기 위한 : (Xi - mean(X)) / sd(X)
+ 정규화 : (Xi - mean(x)) / (max(X) - min(X))
+ .... 이 외에도 종류는 많으나 일반적인 상황에선 두 개로 충분하다.

In [ ]:
#### StandardScaler

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df

print("평균:", df.mean())
print("분산:", df.var())

평균: sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64
분산: sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(df)
iris_scaled= std.transform(df)

# 데이터프레임으로
iris_scaled = pd.DataFrame(iris_scaled, columns = iris.feature_names)
iris_scaled

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444
...,...,...,...,...
145,1.038005,-0.131979,0.819596,1.448832
146,0.553333,-1.282963,0.705921,0.922303
147,0.795669,-0.131979,0.819596,1.053935
148,0.432165,0.788808,0.933271,1.448832


In [ ]:
print("평균:", iris_scaled.mean())
print("분산:", iris_scaled.var())

평균: sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
분산: sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


In [ ]:
# MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df)
iris_scaled = scaler.transform(df)

df_scaled = pd.DataFrame(iris_scaled, columns=iris.feature_names)
df_scaled

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667
...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667
146,0.555556,0.208333,0.677966,0.750000
147,0.611111,0.416667,0.711864,0.791667
148,0.527778,0.583333,0.745763,0.916667


In [ ]:
print("최소값:", df_scaled.min())
print("최대값:", df_scaled.max())

최소값: sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64
최대값: sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


In [ ]:
#### Scaling 할 때 주의해야할 점
#### target 값도 scaling 해야하는데...

train_data = np.arange(0, 11).reshape(-1, 1)
test_data = np.arange(0, 6).reshape(-1, 1)

print(train_data)
print(test_data)

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]


In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data)
train_scale = scaler.transform(train_data)

print("원본:", train_data.reshape(-1))
print("수정:", train_scale.reshape(-1))

## 테스트 데이터도 스케일링 해줘야한다.
## 그런데, 여기서 문제 발생한다.
## 같은 비율로 적용되어야 하는데 자체적으로 하면 그렇지 못함

# scaler.fit(test_data) 이걸 하지 않으면 된다. fit에 비율이 적용됨. 그러니 fit을 하지 않고 transform에 바로 test_data를 넘겨준다.
test_scale = scaler.transform(test_data)
print("원본:", test_data.reshape(-1))
print("수정:", test_scale.reshape(-1))


원본: [ 0  1  2  3  4  5  6  7  8  9 10]
수정: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
원본: [0 1 2 3 4 5]
수정: [0.  0.1 0.2 0.3 0.4 0.5]
